<a href="https://colab.research.google.com/github/Arijit-1988/Terraform_IaC/blob/main/Terraform_Modules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Create a EC2_Module_Project Folder
#### 1.1 Within it there will be **main.tf** file and along with it **ec2_module** folder.

### 1.1 Inside ec2_module folder we will have ec2_module.tf



## Under ***main.tf***


In [ ]:
provide aws {
    region = "us-east-1"
    profile = "arijit-aws"
}

module "ec2_module" {
    source = "./ec2_module"           # directing to "ec2_module.tf" in ec2_module folder.
}


# Terminal : terraform init     terraform plan

### Under ***ec2_module.tf***

In [ ]:
variable "vpc-id" {
    type = 'string'
    default = "172.16.0.0/16"         # Default VPC id from AWS console
}

# Or we can write it in this way

# resource "aws_vpc" "my_vpc" {
#   cidr_block = "172.16.0.0/16"
#   tags = {
#     Name = "tf-example"
#   }
# }

# Security Group

resource "aws_security_group" "Custom_SG" {
  name        = "Custom_SG"
  description = "SSH into EC2 Instance Through port 22"
  vpc_id      = "${var.vpc-id}"         # The variable VPC "vpcid" created above 

  ingress {
    description = "TLS from VPC"
    from_port   = 22
    to_port     = 22
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }

  egress {
    from_port   = 0
    to_port     = 0
    protocol    = "-1"
    cidr_blocks = ["0.0.0.0/0"]
  }

  #tags = {
  #  Name = "allow_tls"
  #}
}

# Creating a Variable for AMI from us-east-1

variable "AMI_us-east-1" {
    default = "ami-005e54dee72cc1d00"
}

# AWS Key pair

resource "aws_key_pair" "MM_PEM" {
  key_name   = "MM_PEM"
  public_key = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQD3F6tyPEFEzV0LX3X8BsXdMsQz1x2cEikKDEY0aIj41qgxMCP/iteneqXSIFZBp5vizPvaoIR3Um9xK7PGoW8giupGn+EPuxIA4cDM4vzOqOkiMPhz5XK0whEjkVzTo4+S0puvDZuwIsdiW9mxhJc7tgBNL0cYlWSYVkz4G/fslNfRPW5mYAM49f4fhtxPb5ok4Q2Lg9dPKVHO/Bgeu5woMc7RY0p1ej6D4CKFE6lymSDJpW0YHX/wqE9+cfEauh7xZcG0q9t2ta6F6fmX0agvpFyZo8aFbXeUBr7osSCJNgvavWbM/06niWrOvYX2xwWdhXmXSrbX8ZbabVohBK41 email@example.com"
}

# Creating EC2 Instance

resource "aws_instance" "EC@_0102_2020" {
  ami = "${var.AMI_us-east-1}"       # Referring to AMI variable in us-east-1
  instance_type = "t2.micro"

  key_name = "${var.aws_key_pair}"

  vpc_security_group_ids = "${aws_security_group.Custom_SG.id}"       # Note: Instances in a **VPC**, use vpc_security_group_ids.
  
}

# Input To Modules


### 1.1 Gonna create another folder **Security Group** folder

#### ***sg_module.tf***